In [46]:
import pandas as pd
import re
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
import wordcloud
from sklearn.preprocessing import OneHotEncoder
import matplotlib.animation as animation
import nltk
from myfunctions import *
sns.set()

In [8]:
tokens = pd.read_csv('All Data/Raw Journey Data/TokensUntil16May2020.csv')

In [9]:
topics = pd.read_csv('All Data/Raw Journey Data/Entry_topics_Until16May2020.csv')
datadf = pd.read_csv(r'All Data/Raw Journey Data/Until16May2020(all jsons to csv).csv')

In [10]:
datadf = datadf.set_index('id')

In [11]:
datadf.head(4)

,text,date_modified,date_journal,address,lat,lon,mood,timezone,favourite,weather,photos,tags,no_punc,is_personal,cleaned_text
id,,,,,,,,,,,,,,,
1439137473084-3fc756cc3f8d6404,"The first thing I saw ,when I entered her room...",2017-08-31 01:44:57,2015-08-09 11:24:33,NaN,1.7976931348623157e+308,1.7976931348623157e+308,0,NaN,False,"{'id': -1, 'degree_c': 1.7976931348623157e+308...",[],"['personal', 'published']",The first thing I saw when I entered her room ...,1,"The first thing I saw , when I entered her roo..."
1439198245820-3fda14add91e3f20,Philosophy\n\n\n\n\n\n\nIt surprises me..hw u ...,2017-08-22 22:08:05,2015-08-10 04:17:25,NaN,1.7976931348623157e+308,1.7976931348623157e+308,0,NaN,False,"{'id': -1, 'degree_c': 1.7976931348623157e+308...",[],['personal'],Philosophy It surprises me hw u so effortlessl...,1,Philosophy It surprises me ... hw you so effor...
1439198461993-3fefd6db711e0342,Me \n\n\nWrite fantasies/inaginary dream world...,2017-10-08 12:24:49,2015-08-10 04:21:01,NaN,1.7976931348623157e+308,1.7976931348623157e+308,0,NaN,False,"{'id': -1, 'degree_c': 1.7976931348623157e+308...",[],['me'],Me Write fantasies inaginary dream world stori...,0,Me Write fantasies / inaginary dream world sto...
1439575035515-3feb330512548cbd,"The dwarfs garden\n\nHe had a garden ,that lit...",2016-04-29 05:23:44,2015-08-14 12:57:15,NaN,1.7976931348623157e+308,1.7976931348623157e+308,0,NaN,False,"{'id': -1, 'degree_c': 1.7976931348623157e+308...",[],['published'],The dwarfs garden He had a garden that little ...,0,"The dwarfs garden He had a garden , that littl..."


In [12]:
tokens

,id,pos,lemm,word,num,lower,pos_sep
0,1439137473084-3fc756cc3f8d6404,"('The', 'DT')",The,The,0,the,DT
1,1439137473084-3fc756cc3f8d6404,"('first', 'JJ')",first,first,1,first,JJ
2,1439137473084-3fc756cc3f8d6404,"('thing', 'NN')",thing,thing,2,thing,NN
3,1439137473084-3fc756cc3f8d6404,"('I', 'PRP')",I,I,3,i,PRP
4,1439137473084-3fc756cc3f8d6404,"('saw', 'VBD')",saw,saw,4,saw,VBD
...,...,...,...,...,...,...,...
329443,1572234096800-3fe09f3bcb856ceb,NaN,NaN,NaN,329443,NaN,NaN
329444,1572394588516-3fe449570c8486e7,NaN,NaN,NaN,329444,NaN,NaN
329445,1573136070396-3feded4b7c9ee046,NaN,NaN,NaN,329445,NaN,NaN
329446,1583630207712-3fddf8c02150a606,NaN,NaN,NaN,329446,NaN,NaN


In [13]:
def bigrammer (text):
    bigrm = list(nltk.bigrams(nltk.word_tokenize(text)))
    return(list(map(lambda x:' '.join(x), bigrm)))

In [14]:
bigrams = datadf[['cleaned_text']].fillna('')

In [15]:
bigrams['cleaned_text'] = bigrams['cleaned_text'].apply(no_stop).apply(no_punc)

In [16]:
bigrams['bigrams'] = bigrams['cleaned_text'].apply(bigrammer)

In [18]:
allbgrms = bigrams.bigrams.explode().to_frame().dropna()
allbgrms['bigrams'] = allbgrms['bigrams'].str.lower()
allbgrms

,bigrams
id,
1439137473084-3fc756cc3f8d6404,first thing
1439137473084-3fc756cc3f8d6404,thing saw
1439137473084-3fc756cc3f8d6404,saw entered
1439137473084-3fc756cc3f8d6404,entered room
1439137473084-3fc756cc3f8d6404,room dark
...,...
1589680083127-3fd337d30df0a5ce,favorite stories
1589680083127-3fd337d30df0a5ce,stories q
1589680083127-3fd337d30df0a5ce,q good


In [17]:
bigrams

,cleaned_text,bigrams
id,,
1439137473084-3fc756cc3f8d6404,first thing saw entered room dark curtains dra...,"[first thing, thing saw, saw entered, entered ..."
1439198245820-3fda14add91e3f20,Philosophy surprises hw effortlessly make top ...,"[Philosophy surprises, surprises hw, hw effort..."
1439198461993-3fefd6db711e0342,Write fantasies inaginary dream world stories ...,"[Write fantasies, fantasies inaginary, inagina..."
1439575035515-3feb330512548cbd,dwarfs garden garden little boy Guarded roses ...,"[dwarfs garden, garden garden, garden little, ..."
1440393646546-3fdb2bc02b9f8020,Brigadier Suraj singh name brigadier suraj sin...,"[Brigadier Suraj, Suraj singh, singh name, nam..."
...,...,...
1589599203833-3fccc2487ff20d88,really work well introverts prefer close frien...,"[really work, work well, well introverts, intr..."
1589599469199-3fc8b4cdb6d4b0d4,Jodi Thor Dak Shoone Ko Na Ashi Tobe Ekla Chol...,"[Jodi Thor, Thor Dak, Dak Shoone, Shoone Ko, K..."
1589603430128-3fdcc9774f183676,really since told girl love,"[really since, since told, told girl, girl love]"


In [19]:
allbgrms.bigrams.value_counts()

can not            282
https youtu         90
_ _                 72
one day             59
first time          58
                  ... 
badge said           1
clarify central      1
make educated        1
sleep lack           1
place bitch          1
Name: bigrams, Length: 109131, dtype: int64

In [20]:
# allbgrms.to_csv(r'All Data/Raw Journey Data/Bigrams_Until16May2020.csv')

In [35]:
%%time
temp = tokens.pos_sep.value_counts().to_frame()
temp['proportions']=(temp.pos_sep/temp.pos_sep.sum())*100
temp

CPU times: user 35.8 ms, sys: 33.6 ms, total: 69.4 ms
Wall time: 109 ms


,pos_sep,proportions
NN,42141,12.795397
PRP,30155,9.156052
IN,27139,8.240295
.,23955,7.273528
DT,21097,6.405745
JJ,19011,5.772366
RB,18461,5.605368
VB,15088,4.581214
VBP,13322,4.044998
NNP,13291,4.035586


In [43]:
pd.set_option('display.max_rows', 100)
temp = tokens[tokens.pos_sep=='NN']['lower'].value_counts()
temp = temp[temp>1]
temp[:100]

time            676
life            538
way             465
something       459
world           392
day             384
someone         376
person          371
everything      319
man             281
lot             270
book            270
thing           265
everyone        245
anything        218
today           211
hai             206
guy             191
year            183
nothing         183
adis            161
love            153
place           144
mind            144
night           144
https           144
story           142
/               141
class           139
one             137
room            125
head            122
part            122
work            119
face            119
hand            118
movie           115
power           113
friend          113
heart           113
school          112
stuff           107
*               107
end             105
front           105
home            101
boy             101
kind             99
money            94
name             93


In [67]:
# tokens[tokens.lower=='hai'].groupby('id').first()

In [66]:
# datadf.cleaned_text.loc[]